In [1]:
import torch
import cv2
import numpy as np
from torchvision import transforms
from scipy.ndimage import maximum_filter

### Load CSRNet Model

In [4]:
import torch
import torch.nn as nn

cfg = {
    'frontend': [64,64,'M',128,128,'M',256,256,256,'M',512,512,512],
    'backend' : [512,512,512,256,128,64]
}

def make_layers(cfg_list, in_channels=3, dilation=False):
    layers = []
    for v in cfg_list:
        if v == 'M':
            layers.append(nn.MaxPool2d(2,2))
        else:
            d = 2 if dilation else 1
            layers += [
                nn.Conv2d(in_channels, v, kernel_size=3, padding=d, dilation=d),
                nn.ReLU(inplace=True)
            ]
            in_channels = v
    return nn.Sequential(*layers)

class CSRNet(nn.Module):
    def __init__(self):
        super(CSRNet, self).__init__()
        
        self.frontend = make_layers(cfg['frontend'], in_channels=3, dilation=False)
        self.backend  = make_layers(cfg['backend'], in_channels=512, dilation=True)
        self.output   = nn.Conv2d(64, 1, kernel_size=1)

    def forward(self, x):
        x = self.frontend(x)
        x = self.backend(x)
        x = self.output(x)
        return x   

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = CSRNet()
model.load_state_dict(torch.load("models/CSRNet_best.pth", map_location=device))
model.to(device)
model.eval()

CSRNet(
  (frontend): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilatio

### Preprocess Frame

In [7]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

def preprocess(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = transform(img)
    return img.unsqueeze(0)


### Predict Density Map

In [8]:
def predict_density(frame):
    inp = preprocess(frame).to(device)
    with torch.no_grad():
        density = model(inp)
    density = density.squeeze().cpu().numpy()
    return density


### Convert Density Map to Head Points

In [9]:
def density_to_points(density, threshold=0.2):
    density_norm = density / density.max()
    peaks = (density_norm == maximum_filter(density_norm, size=5))
    peaks &= (density_norm > threshold)

    points = np.column_stack(np.where(peaks))
    return points


### Visualize

In [10]:
def visualize_points(frame, points):
    vis = frame.copy()
    for (y, x) in points:
        cv2.circle(vis, (x, y), 3, (0, 0, 255), -1)
    return vis


### Process All Frames 

In [11]:
import os

frames_dir = "frames/"
save_dir   = "pseudo_labels/"
os.makedirs(save_dir, exist_ok=True)

for img_name in os.listdir(frames_dir):
    img_path = os.path.join(frames_dir, img_name)
    img = cv2.imread(img_path)

    density = predict_density(img)
    points = density_to_points(density)

    np.save(os.path.join(save_dir, img_name.replace(".jpg", ".npy")), points)

    vis = visualize_points(img, points)
    cv2.imwrite("visualized_" + img_name, vis)


In [13]:
import os
import cv2
import numpy as np
from scipy.ndimage import gaussian_filter

frames_dir = "frames/"
points_dir = "pseudo_labels/"
save_dir   = "dataset/density/"

os.makedirs(save_dir, exist_ok=True)

def generate_density_map(img_shape, points, sigma=4):
    h, w = img_shape[:2]
    density = np.zeros((h, w), dtype=np.float32)

    for y, x in points:
        if y < h and x < w:
            density[int(y), int(x)] = 1

    density = gaussian_filter(density, sigma=sigma)
    return density

def downsample_density(density, factor=8):
    h, w = density.shape
    density_ds = cv2.resize(
        density,
        (w // factor, h // factor),
        interpolation=cv2.INTER_CUBIC
    )
    density_ds *= factor * factor
    return density_ds

for name in os.listdir(frames_dir):
    if not name.endswith(".jpg"):
        continue

    img = cv2.imread(os.path.join(frames_dir, name))
    points = np.load(os.path.join(points_dir, name.replace(".jpg", ".npy")))

    density = generate_density_map(img.shape, points)
    density = downsample_density(density)

    np.save(os.path.join(save_dir, name.replace(".jpg", ".npy")), density)

print(" Density maps generated successfully")


 Density maps generated successfully
